Simple Iterative Methods
=

In this chapter we learn

* The Richardson Iteration
* The Gradient Method
* Using Preconditioners

We are given a linear system of equations

$$
A x = b.
$$

The matrix $A$ is so large such that direct elimination is not a good option. 
Although this section applies to linear systems in general, we think 
of equations arising from finite element discretization. Other numerical methods for partial differential equations lead to similar systems. Matrices from graph networks are another example.

The situation is advantageous if the matrix is
* symmetric:  $$A = A^T$$
* positive definite: $$x^T A x > 0 \qquad \forall \, 0 \neq x \in R^n$$

We call symmetric and positive definite matrices SPD.

We do not assume that the matrix is stored as a dense matrix, i.e. all $n^2$ entries are stored in a two dimensional array.

We only assume that the matrix-vector product

$$
y := A * x
$$

is computationally available. 

Typically, the algorithmic complexity of the matrix-vector product is $O(n)$. 

Often one stores the matrix in a sparse matrix format, such as the CSR (Compressed Sparse Row) format. https://de.wikipedia.org/wiki/Compressed_Row_Storage https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html


More general, the matrix-vector product can be any linear operator. We can think of the sum or product of linear operators, multiplying with the inverse of the diagonal, or solving a triangular (sparse) system by forward/backward substitution

The Richardson Iteration
---
also called simple iteration is the fixed point iteration

$$
x^{k+1} := x^k + \alpha \, (b - A x^k)
$$

with an arbitrary starting value $x^0 \in R^n$, and a properly cosen damping paramter $\alpha$.

The solution $x^\ast$ is a fixed point of the iteration (since $b - A x^\ast = 0$).

If we define the error as
$$
e^k = x^k - x^\ast,
$$

the error propagation from one step to the next is

\begin{eqnarray*}
e^{k+1} & = & x^{k+1} - x^\ast = x^k + \alpha (b - A x^k) - x^\ast \\
& = & x^k - x^\ast + \alpha A (x^\ast - x^k) = (I - \alpha A) (x^k - x^\ast)
\end{eqnarray*}

This means the new error is obtained from the old error by the error propagation matrix

$$
e^{k+1} = (I - \alpha A) \, e^k
$$

Two strategies to verify convergence are:

* prove that the convergence radius 

$$
\rho(I - \alpha A) := \max_{\lambda \in \sigma(I - \alpha A)} |\lambda| < 1
$$

* find some norm $\| \cdot \|$ such that the matrix norm (=operator norm)

$$
\| I - \alpha A \| := \sup_{x \in R^n} \frac{ \| (I - \alpha A) x \| }{ \| x \| } < 1
$$

The first one, $\rho < 1$, only provides asymptotic convergence. This is easily proven if A is diagonizable, i.e. it features a full set of eigenvectors $z^j$ and eigenvalues $\lambda_j$. Expand the initial error as
$$
e^0 = \sum_j e^0_j z^j,
$$
then
$$
e^k = \sum_j (1-\alpha \lambda_j)^k e^0_j z^j
$$
and
$$
\| e^k \| \leq \rho^k  \sum_j | e^0_j | \| z^j \|
$$
This means $\| e^k \| \leq C \rho^k$, the the error does not have to decrease monotonically.

However, if $\| I - \alpha A \| < 1$, then
$$
\| e^k \| \leq \| I - \alpha A \| \, \| e^k \|,
$$
and the error decreases in every interation step. Note that the matrix norm is the operator norm generated by the vector norm.

Some facts:
* If the norm $\| \cdot \|$ is generated by an inner product $\left< \cdot, \cdot \right>$ (parallelogram identity), and $M$ is some self adjoint matrix with respect to this inner product, i.e.
$$
\left< M x, y \right>  = \left< x, M y \right>,
$$
  then $\rho(M) = \| M \|$

  If $\left< \cdot , \cdot \right>$ is the Euklidean inner product, then $M$ is self-adjoint exactly when $M$ is symmetric.
  
* Every operator operator norm is bounded by the spectral radius. There exists some norm such that the operator norm is arbitrary close to the spectral radius, i.e.
$$
\rho(M) = \sup_{ \text{norms} \| \cdot \| } \| M \|
$$

Optimizing the relaxation parameter $\alpha$
---
Let $A$ be SPD, and let $\sigma(A) = \{ \lambda_i \in R \}$ with $0 < \lambda_1 \leq \lambda_2 \ldots \leq \lambda_n$.

Then the eigenvalues of $M = I - \alpha A$ are $\{ 1 - \alpha \lambda_i  \}$. 

Whenever we choose 
$$
0 < \alpha < \frac{2}{\lambda_n}
$$
we obtain $\rho(M) < 1$ and a convergent iteration.


The spectral radius of $M$ is 

$$
\rho(M) = \max \{ | 1 - \alpha \lambda_i| \}  = 
\max \{ 1 - \alpha \lambda_1, - (1-\alpha \lambda_n) \}
$$

The maximum is minimized if we choose $\alpha$ optimally such that
$$
1 - \alpha \lambda_1 = - (1 - \alpha \lambda_n),
$$
i.e.
$$
\alpha_{\text{opt}} = \frac{2}{\lambda_1 + \lambda_n} 
$$
leading to the optimal convergence rate

$$
\rho_{\text{opt}} = \frac{\lambda_n - \lambda_1}{\lambda_n+\lambda_1}
\approx 1 - 2 \frac{\lambda_1}{\lambda_n} = 1 - \frac{2}{\kappa(A)}
$$

with the condition number $\kappa(A) = \lambda_n(A) / \lambda_1(A)$.

Experiments with the Richardson iteration
---

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [ ]:
fes = H1(mesh, order=1)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+10*u*v*dx).Assemble()
f = LinearForm(x*y*v*dx).Assemble()
gfu = GridFunction(fes)

we determine (an approximation to) the largest eigenvalue by a few steps of the power iteration:

In [ ]:
hv = gfu.vec.CreateVector()
hv2 = gfu.vec.CreateVector()
hv.SetRandom()
hv.data /= Norm(hv)
for k in range(100):
    hv2.data = a.mat * hv
    rho = Norm(hv2)
    print (rho)
    hv.data = 1/rho * hv2

In [ ]:
alpha = 1 / rho
res = f.vec.CreateVector()
gfu.vec[:] = 0
for k in range(1000):
    res.data = f.vec - a.mat * gfu.vec
    print ("iteration", k, "res=", Norm(res))
    gfu.vec.data += alpha * res

In [ ]:
from ngsolve.webgui import Draw
Draw (gfu)